<a href="https://colab.research.google.com/github/CAR-Driving/yoloOnGoogleColab/blob/master/Kitti_Database_Downloding_and_Converting_to_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## inserting the KITTI - Object Detection
Detect and localize multiple Classes in images
 
KITTI Object Detection with Bounding Boxes – Taken from the benchmark suite from the Karlsruhe Institute of Technology, this dataset consists of images from the object detection section of that suite. This image dataset includes over 14,000 images made up of 7,518 testing images and 7,481 training images with bounding boxes labels in a separate file.
 
https://www.kaggle.com/twaldo/kitti-object-detection

# First of all get The **kaggle.json** file from this link, as you can see blowe:


```
*   https://www.kaggle.com/<your Account)/account

```



![enter image description here](https://i.stack.imgur.com/Uz4KW.png)

Then Upload the **kaggle.json** by Blow code:


In [ ]:
from google.colab import files
files.upload()
 
!mkdir -p ~/.kaggle
 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
 
 
!pip install kaggle 
#!sudo apt install kaggle 
!kaggle datasets download -d twaldo/kitti-object-detection

## Location of the Images:
![enter image description here](https://i.stack.imgur.com/BBEfj.jpg)






In [ ]:
!ls
!unzip kitti-object-detection.zip

#  convert from data in [KITTI](https://www.kaggle.com/twaldo/kitti-object-detection) format to [Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/htmldoc/index.html) format:

In [24]:


# Script to convert yolo annotations to voc format
import os
import xml.etree.cElementTree as ET
from PIL import Image
from math import floor

###########################
#set directories and class names
###########################
ANNOTATIONS_DIR_PREFIX = "/content/kitti_single/training/label_2"
IMAGE_DIR_PREFIX ="/content/kitti_single/training/image_2"
imgExt = "png"
imgChnls = 3 #RGB:3 ; Grayscale:1

DESTINATION_DIR =  '/content/kitti_single/kitti_xml'

CLASS_MAPPING = {
    '0': 'Pedestrian',
    '1': 'Misc',
    '2': 'Car',
    '3': 'DontCare'
}

###########################

def create_root(file_prefix, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "filename").text = "{}.{}".format(file_prefix,imgExt)
    ET.SubElement(root, "folder").text = "{}/{}.{}".format(IMAGE_DIR_PREFIX,file_prefix,imgExt)
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = str(imgChnls)
    return root


def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = voc_label[0]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label[1])
        ET.SubElement(bbox, "ymin").text = str(voc_label[2])
        ET.SubElement(bbox, "xmax").text = str(voc_label[3])
        ET.SubElement(bbox, "ymax").text = str(voc_label[4])
    return root


def create_file(file_prefix, width, height, voc_labels):
    root = create_root(file_prefix, width, height)
    root = create_object_annotation(root, voc_labels)
    tree = ET.ElementTree(root)
    

    tree.write("{}/{}.xml".format(DESTINATION_DIR, file_prefix))


def read_file(file_path):
    file_prefix = file_path.split(".txt")[0]
    image_file_name = "{}.{}".format(file_prefix,imgExt)
    img = Image.open("{}/{}".format(IMAGE_DIR_PREFIX, image_file_name))
    print(img)

    w, h = img.size
    prueba = "{}/{}".format(ANNOTATIONS_DIR_PREFIX, file_path)
    print(prueba)
    with open(prueba) as file:
        lines = file.readlines()
        voc_labels = []
        for line in lines:	
            voc = []
            line = line.strip()
            data = line.split()
            voc.append(CLASS_MAPPING.get(data[0]))
            bbox_width = float(data[3]) * w
            bbox_height = float(data[4]) * h
            center_x = float(data[1]) * w
            center_y = float(data[2]) * h
            voc.append(floor(center_x - (bbox_width / 2)))
            voc.append(floor(center_y - (bbox_height / 2)))
            voc.append(floor(center_x + (bbox_width / 2)))
            voc.append(floor(center_y + (bbox_height / 2)))
            voc_labels.append(voc)
        create_file(file_prefix, w, h, voc_labels)
    print("Processing complete for file: {}".format(file_path))


def start():
    if not os.path.exists(DESTINATION_DIR):
        os.makedirs(DESTINATION_DIR)
    for filename in os.listdir(ANNOTATIONS_DIR_PREFIX):
        if filename.endswith('txt'):
            try:
                PathFileName = "{}/{}".format(ANNOTATIONS_DIR_PREFIX, filename)
                if os.stat(PathFileName).st_size > 0:
                    print("Si")
                    read_file(filename) 
            except:
                print("No")         
            
        else:
            print("Skipping file: {}".format(filename))


if __name__ == "__main__":
    start()


Streaming output truncated to the last 5000 lines.
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A93720A90>
/content/kitti_single/training/label_2/005216.txt
Processing complete for file: 005216.txt
Si
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A93720F28>
/content/kitti_single/training/label_2/005322.txt
Processing complete for file: 005322.txt
Si
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A93720E10>
/content/kitti_single/training/label_2/000581.txt
Processing complete for file: 000581.txt
Si
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A93720F98>
/content/kitti_single/training/label_2/001305.txt
Processing complete for file: 001305.txt
Si
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A93720C88>
/content/kitti_single/training/label_2/002771.txt
Processing complete for file: 002771.txt
Si
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1242x375 at 0x7F7A937

In [ ]:
%cd /content/kitti_single
import glob
var1=glob.glob('**/*.png', recursive=True)
string = '/content/kitti_single/'
my_new_list = [string + x for x in var1]
print(my_new_list)
print ('\n'.join([i for i in my_new_list[1:]]))
photo2=('\n'.join([i for i in my_new_list[1:]]))
text_file = open("/content/kitti_single/train.txt", "w")
text_file.write(photo2)
text_file.close()

In [ ]:
# %cd /content/
# !git clone https://github.com/umautobots/vod-converter

In [ ]:
# # !mkdir /content/kitti_single/XML_converted
# !python /content/vod-converter/vod_converter/main.py --from kitti --from-path /content/kitti_single/training --to voc --to-path /content/kitti_single/XML_converted

In [5]:
from lxml import etree
from PIL import Image
import csv
import os

# (NOTICE #1)
# If you're OS is Mac, there's a case when '.DS_Store' file is  automatically created.
# In that case, you have to remove '.DS_Store' file through the terminal.
# Ref : http://leechoong.com/posts/2018/ds_store/
# (NOTICE #2)
# Change this path variable
IMG_PATH = "/content/kitti_single/training/image_2"
fw = os.listdir(IMG_PATH)
# path of save xml file
save_path = '/content/kitti_single/testing/' # keep it blank

# txt_folder is txt file root that using darknet rectbox
txt_folder = '/content/kitti_single/training/label_2'

# (NOTICE #3)
# Change this labels
labels = ['dog_two_stand', 'dog_four_stand', 'dog_sit', 'dog_lying']
global label
label = ''

def csvread(fn):
    with open(fn, 'r') as csvfile:
        list_arr = []
        reader = csv.reader(csvfile, delimiter=' ')

        for row in reader:
            list_arr.append(row)
    return list_arr


def convert_label(txt_file):
    global label
    for i in range(len(labels)):
        if txt_file[0] == str(i):
            label = labels[i]
            return label

    return label

# core code = convert the yolo txt file to the x_min,x_max...


def extract_coor(txt_file, img_width, img_height):
    x_rect_mid = float(txt_file[1])
    y_rect_mid = float(txt_file[2])
    width_rect = float(txt_file[3])
    height_rect = float(txt_file[4])

    x_min_rect = ((2 * x_rect_mid * img_width) - (width_rect * img_width)) / 2
    x_max_rect = ((2 * x_rect_mid * img_width) + (width_rect * img_width)) / 2
    y_min_rect = ((2 * y_rect_mid * img_height) -
                  (height_rect * img_height)) / 2
    y_max_rect = ((2 * y_rect_mid * img_height) +
                  (height_rect * img_height)) / 2

    return x_min_rect, x_max_rect, y_min_rect, y_max_rect


for line in fw:
    root = etree.Element("annotation")

    # try debug to check your path
    img_style = IMG_PATH.split('/')[-1]
    img_name = line
    image_info = IMG_PATH + "/" + line
    img_txt_root = txt_folder + "/" + line[:-4]
    # print(img_txt_root)
    txt = ".txt"

    txt_path = img_txt_root + txt
    # print(txt_path)
    txt_file = csvread(txt_path)
    ######################################

    # read the image  information
    img_size = Image.open(image_info).size

    img_width = img_size[0]
    img_height = img_size[1]
    img_depth = Image.open(image_info).layers
    ######################################

    folder = etree.Element("folder")
    folder.text = "%s" % (img_style)

    filename = etree.Element("filename")
    filename.text = "%s" % (img_name)

    path = etree.Element("path")
    path.text = "%s" % (IMG_PATH)

    source = etree.Element("source")
    ##################source - element##################
    source_database = etree.SubElement(source, "database")
    source_database.text = "Unknown"
    ####################################################

    size = etree.Element("size")
    ####################size - element##################
    image_width = etree.SubElement(size, "width")
    image_width.text = "%d" % (img_width)

    image_height = etree.SubElement(size, "height")
    image_height.text = "%d" % (img_height)

    image_depth = etree.SubElement(size, "depth")
    image_depth.text = "%d" % (img_depth)
    ####################################################

    segmented = etree.Element("segmented")
    segmented.text = "0"

    root.append(folder)
    root.append(filename)
    root.append(path)
    root.append(source)
    root.append(size)
    root.append(segmented)

    for ii in range(len(txt_file)):
        label = convert_label(txt_file[ii][0])
        x_min_rect, x_max_rect, y_min_rect, y_max_rect = extract_coor(
            txt_file[ii], img_width, img_height)

        object = etree.Element("object")
        ####################object - element##################
        name = etree.SubElement(object, "name")
        name.text = "%s" % (label)

        pose = etree.SubElement(object, "pose")
        pose.text = "Unspecified"

        truncated = etree.SubElement(object, "truncated")
        truncated.text = "0"

        difficult = etree.SubElement(object, "difficult")
        difficult.text = "0"

        bndbox = etree.SubElement(object, "bndbox")
        #####sub_sub########
        xmin = etree.SubElement(bndbox, "xmin")
        xmin.text = "%d" % (x_min_rect)
        ymin = etree.SubElement(bndbox, "ymin")
        ymin.text = "%d" % (y_min_rect)
        xmax = etree.SubElement(bndbox, "xmax")
        xmax.text = "%d" % (x_max_rect)
        ymax = etree.SubElement(bndbox, "ymax")
        ymax.text = "%d" % (y_max_rect)
        #####sub_sub########

        root.append(object)
        ####################################################

    file_output = etree.tostring(root, pretty_print=True, encoding='UTF-8')
    # print(file_output.decode('utf-8'))
    ff = open('%s.xml' % (img_name[:-4]), 'w', encoding="utf-8")
    ff.write(file_output.decode('utf-8'))

AttributeError: ignored

In [ ]:
# import pathlib

# list(pathlib.Path('/content/kitti_single').glob('*.png'))

In [ ]:
def create_dataset(clothing, categories, dataset_type):

  images_path = Path(f"clothing/images/{dataset_type}")
  images_path.mkdir(parents=True, exist_ok=True)

  labels_path = Path(f"clothing/labels/{dataset_type}")
  labels_path.mkdir(parents=True, exist_ok=True)

  for img_id, row in enumerate(tqdm(clothing)):

    image_name = f"{img_id}.jpeg"

    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert("RGB")

    img.save(str(images_path / image_name), "JPEG")

    label_name = f"{img_id}.txt"

    with (labels_path / label_name).open(mode="w") as label_file:

      for a in row['annotation']:

        for label in a['label']:

          category_idx = categories.index(label)

          points = a['points']
          p1, p2 = points

          x1, y1 = p1['x'], p1['y']
          x2, y2 = p2['x'], p2['y']

          bbox_width = x2 - x1
          bbox_height = y2 - y1

          label_file.write(
            f"{category_idx} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
          )

# Install Yolo:

https://pjreddie.com/darknet/yolo/


In [ ]:

import cv2, os
import matplotlib.pyplot as plt
%matplotlib inline
  

In [ ]:

!ls
!cd /content
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/
% cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt update
!apt-get install libopencv-dev

kaggle.json  kitti-object-detection.zip  kitti_single  readme.txt  sample_data
Cloning into 'darknet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 14367 (delta 2), reused 4 (delta 1), pack-reused 14358
Receiving objects: 100% (14367/14367), 13.08 MiB | 19.47 MiB/s, done.
Resolving deltas: 100% (9766/9766), done.
/content/darknet
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.do

## `Compile and Configure`


* Compile YOLO
* Download YOLO weights



In [ ]:
!make &> compile.log

In [ ]:
!wget https://free.keep.sh/ArnjpAAwgQgzAe89/yolov3.weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights



Mounted at /content/drive/
cp: -r not specified; omitting directory '/content/drive/My Drive/Colab Notebooks/yolo'


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# !cp -r '/content/drive/My Drive/Colab Notebooks/yolo/yolov3.weights' '/content/darknet/'

In [ ]:
#https://colab.research.google.com/drive/1lTGZsfMaGUpBG4inDIQwIJVW476ibXk_#scrollTo=v3nkYzWwMuBk



# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!wget http://people.cs.uchicago.edu/~kauffman/nvidia/cudnn/cudnn-9.0-linux-x64-v7.6.5.32.tgz
!tar -xzvf /content/darknet/cudnn-9.0-linux-x64-v7.6.5.32.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.5
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 5
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


In [ ]:
!pip install mxnet-cu80

     |████████████████████████████████| 372.1MB 30kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
%cd /content/darknet
!./darknet detector test coco.names yolov3.cfg yolov3.weights -thresh 0.24 -dont_show < /content/kitti_single/train.txt > result.txt

/content/darknet
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:325: error: Assertion `0' failed.
/bin/bash: line 1:  3457 Aborted                 (core dumped) ./darknet detector test coco.names yolov3.cfg yolov3.weights -thresh 0.24 -dont_show < /content/kitti_single/train.txt > result.txt


In [ ]:
%cd /content/darknet

!./darknet detector test cfg/voc.data yolo-voc.cfg yolo-voc.weights -dont_show -ext_output < /content/kitti_single/train.txt > result.txt

/content/darknet
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:325: error: Assertion `0' failed.
/bin/bash: line 1:  3463 Aborted                 (core dumped) ./darknet detector test cfg/voc.data yolo-voc.cfg yolo-voc.weights -dont_show -ext_output < /content/kitti_single/train.txt > result.txt


https://github.com/thtrieu/darkflow/blob/master/README.md


![ssa](https://raw.githubusercontent.com/thtrieu/darkflow/master/demo.gif)

In [ ]:
!./flow --imgdir /content/kitti_single/training/image_2/ --model cfg/tiny-yolo.cfg --load bin/tiny-yolo.weights --json

In [ ]:
#https://colab.research.google.com/github/ankitAMD/Darkflow-object-detection/blob/master/custom%20Detectionon%20solar%20panel.ipynb



import cv2  #Open Source Computer Vision is a library which helps in real-time computer vision and used for Image Processing
from darkflow.net.build import TFNet     #Tfnet is a class in build.py python files through which  Translate darknet to tensorflow.
import matplotlib.pyplot as plt          #it is used for data visualization

%config InlineBackend.figure_format = 'svg'

# define the model options and run



# Training the model
# define the model options and run
#This step involves training your model based on dataset and labels you generated*

options = {
    'metaLoad':"built_graph/yolo-4c.meta", 
    'pbLoad':"built_graph/yolo-4c.pb",
    'threshold': 0.07,
    'cpu': 1.0
}


tfnet = TFNet(options)


In [ ]:
!mkdir /content/darknet/dataset_img/
!mkdir /content/darknet/dataset_img/good
!mkdir /content/darknet/dataset_img/Broken


In [ ]:
#import the library opencv
import cv2
#globbing utility.
import glob
#select the path
path = "/content/kitti_single/training/image_2/*.png"
for file in glob.glob(path):
    #print(file)
    filename = file.split('/')[-1]
    print(filename)
    
    img= cv2.imread(file,cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # use YOLO to predict the image
    result = tfnet.return_predict(img)
    length = len(result)
    #print(result)
    print(length)

# pull out some info from the results
    for i in range(0, length):

        tl = (result[i]['topleft']['x'], result[i]['topleft']['y'])
        br = (result[i]['bottomright']['x'], result[i]['bottomright']['y'])
        label = result[i]['label']
     
    
        img = cv2.rectangle(img, tl, br, (200, 10, 40), 2)
        img = cv2.putText(img,label, tl, cv2.FONT_ITALIC, 1, (0,0,0),2)
        for x in result:
            if x['label']== 'Good':
                    #print(x['confidence'])
                status = cv2.imwrite('/home/assetone04/Desktop/Project/Good/' + str(filename), img)   
            elif x['label']== 'snow':
                status = cv2.imwrite('/home/assetone04/Desktop/Project/snow/' +  str(filename) , img)
            elif x['label']== 'Broken':
                status = cv2.imwrite('/home/assetone04/Desktop/Project/Broken/' +  str(filename) , img)
            else :
                status = cv2.imwrite('/home/assetone04/Desktop/Project/Fire/' +  str(filename) , img)
                
         
    
# add the box and label and display it

    plt.imshow(img)
    plt.show()
    print(label)
